<a href="https://colab.research.google.com/github/IBM-EPBL/IBM-Project-16835-1659623726/blob/main/IBM-Project-16835-1659623726/Assignments/Team-Member-1-SaiSriyaV/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Read dataset and do pre-processing

In [3]:
df = pd.read_csv("/spam.csv",delimiter=",",encoding="latin-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [7]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

Create Model and Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [8]:
inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

Compile the Model

In [9]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Train and Fit the Model

In [10]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 11s 277ms/step - loss: 0.3343 - accuracy: 0.8712 - val_loss: 0.2035 - val_accuracy: 0.9768
Epoch 2/10
30/30 [==============================] - 8s 259ms/step - loss: 0.0907 - accuracy: 0.9802 - val_loss: 0.0757 - val_accuracy: 0.9768
Epoch 3/10
30/30 [==============================] - 8s 259ms/step - loss: 0.0471 - accuracy: 0.9855 - val_loss: 0.0603 - val_accuracy: 0.9831
Epoch 4/10
30/30 [==============================] - 8s 259ms/step - loss: 0.0310 - accuracy: 0.9913 - val_loss: 0.0803 - val_accuracy: 0.9800
Epoch 5/10
30/30 [==============================] - 8s 261ms/step - loss: 0.0250 - accuracy: 0.9931 - val_loss: 0.0641 - val_accuracy: 0.9789
Epoch 6/10
30/30 [==============================] - 8s 258ms/step - loss: 0.0201 - accuracy: 0.9952 - val_loss: 0.0696 - val_accuracy: 0.9810
Epoch 7/10
30/30 [==============================] - 8s 273ms/step - loss: 0.0156 - accuracy: 0.9955 - val_loss: 0.1348 - val_accuracy: 0.9747
Epoch

Save The Model

In [11]:
model.save('sms_classifier.h5')

Preprocessing the Test Dataset

In [12]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

Testing the Model

In [13]:
accr = model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 24ms/step - loss: 0.0585 - accuracy: 0.9844


In [14]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.058
  Accuracy: 0.984
